In [ ]:

import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

from unittest import TestCase
TestCase.maxDiff = None


OSM_PATH = "sample.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'(\w|_)+:(\w|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'\?%#$@\,\. \t\r\n]')

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "Ave" : "Avenue",
            "Ave." : "Avenue",
            "Rd" : "Road",
            "Rd." : "Road",
            "RD" : "Road",
            "Blvd" : "Boulevard",
            "Blvd." : "Boulevard",
            "Cir" : "Circle",
            "Cir." : "Circle",
            "Ct" : "Court",
            "Ct." : "Court",
            "Dr" : "Drive",
            "Dr." : "Drive",
            "Pl" : "Place",
            "Pl." : "Place"
            }


SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

def update_name(name, mapping):

    m = street_type_re.search(name)
    other_street_types = []
    if m:
        street_type = m.group()
        if street_type in mapping.keys():
            name = re.sub(street_type_re,mapping[street_type],name)
        else:
            other_street_types.append(street_type)

    return name

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements
    
    node_tags = {}
    
    # YOUR CODE HERE
    if element.tag == 'node':
        for i in element.attrib:
            if i in node_attr_fields:
                node_attribs[i] = element.attrib[i]

        for tag in element.iter('tag'):
            if PROBLEMCHARS.match(tag.attrib['k']):
                pass
            elif LOWER_COLON.match(tag.attrib['k']):
                if tag.attrib['k'] == "addr:street":
                    node_tags['id'] = element.attrib['id']
                    node_tags['key'] = 'street'
                    node_tags['type'] = "addr"
                    node_tags['value'] = update_name(tag.attrib['v'], mapping)
                    tags.append(node_tags.copy())
                else:
                    node_tags['id']=element.attrib['id']
                    node_tags['key']=tag.attrib['k'].split(":",1)[1]
                    node_tags['value']=tag.attrib['v']
                    node_tags['type']=tag.attrib['k'].split(":",1)[0]
                    tags.append(node_tags.copy())
            else:
                node_tags['id']=element.attrib['id']
                node_tags['key']=tag.attrib['k']
                node_tags['value']=tag.attrib['v']
                node_tags['type']='regular'
                tags.append(node_tags.copy())
            
        return {'node': node_attribs, 'node_tags': tags}

    elif element.tag == 'way':

        for i in element.attrib:
            if i in way_attr_fields:
                way_attribs[i]=element.attrib[i]
            
        for way_tag_value in element.iter('tag'):
            if PROBLEMCHARS.match(way_tag_value.attrib['k']):
                pass
            elif LOWER_COLON.match(way_tag_value.attrib['k']):
                if way_tag_value.attrib['k'] == "addr:street":
                    node_tags['id'] = element.attrib['id']
                    node_tags['key'] = 'street'
                    node_tags['type'] = "addr"
                    node_tags['value'] = update_name(way_tag_value.attrib['v'], mapping)
                    tags.append(node_tags.copy())
                else:
                    node_tags['id']=element.attrib['id']
                    node_tags['key']=way_tag_value.attrib['k'].split(":",1)[1]
                    node_tags['value']=way_tag_value.attrib['v']
                    node_tags['type']=way_tag_value.attrib['k'].split(":",1)[0]
                    tags.append(node_tags.copy())
            else:
                node_tags['id']=element.attrib['id']
                node_tags['key']=way_tag_value.attrib['k']
                node_tags['value']=way_tag_value.attrib['v']
                node_tags['type']='regular'
                tags.append(node_tags.copy())
        i=0
        for way_nd_value in element.iter('nd'):
            way_nodes.append({'id':element.attrib['id'], 'node_id':way_nd_value.attrib['ref'], 'position':i})
            i += 1
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}



# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)
    


In [1]:
files = ['nodes.csv','ways.csv', 'ways_tags.csv', 'nodes_tags.csv', 'ways_nodes.csv']
    
for filename in files:
    with open('{}_{}'.format('new',filename), 'w') as f_out:
        for line in open(filename):
            line = line.rstrip()
            if line != '':
                line = line + '\n'
                f_out.write(line)